In [1]:
import numpy as np
import multiprocessing as mp
from scipy.stats import norm, normaltest
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from simulate import simulate_job
from config_plt import config_plt, get_fig_ax, save_my_fig

In [ ]:
mp.set_start_method("spawn", force=True)
T = 10_000
n_reps = 10_000
a_target = 100
n_jobs = 4
ss = np.random.SeedSequence(1)
child_ss = ss.spawn(n_reps)
seeds = [int(s.generate_state(1)[0]) for s in child_ss]
args = [(seed, T, a_target) for seed in seeds]
with mp.Pool(processes=n_jobs) as pool:
    results = list(tqdm(pool.imap_unordered(simulate_job, args, chunksize=1), total=n_reps))

  1%|█▏                                                                             | 145/10000 [00:03<03:26, 47.83it/s]

In [ ]:
zs = np.asarray([r[0] for r in results])
us = np.asarray([r[1] for r in results])
psis = np.asarray([r[2] for r in results])
tauhats = np.asarray([r[3] for r in results])
taus = np.asarray([r[4] for r in results])
equal_groups = np.asarray([r[5] for r in results])
t_targets = np.asarray([r[6] for r in results])

In [ ]:
psis_equal = psis[equal_groups]
psis_equal_mean = np.mean(psis_equal)
print(psis_equal_mean)
psis_unequal = psis[~equal_groups]
psis_unequal_mean = np.mean(psis_unequal)
print(psis_unequal_mean)

In [ ]:
print(normaltest(zs))
fig, ax = get_fig_ax()
ax = sns.kdeplot(zs, ax=ax, bw_method=0.2, label="KDE")
zs_vals = np.linspace(-5, 5, 1000)
ax.plot(zs_vals, norm.pdf(zs_vals), label="N(0, 1)")
ax.set_xlabel("Z")
ax.legend()
save_my_fig(fig, "z")

In [ ]:
us_std = us / np.std(us)
print(normaltest(us_std))
fig, ax = get_fig_ax()
ax = sns.kdeplot(us_std, bw_method=0.2, ax=ax, label="KDE")
zs_vals = np.linspace(-5, 5, 1000)
ax.plot(zs_vals, norm.pdf(zs_vals), label="N(0, 1)")
ax.set_xlabel("U")
ax.legend()
save_my_fig(fig, "u")

In [ ]:
plt.hist(t_targets)

In [ ]:
plt.hist(tauhats)

In [ ]:
plt.hist(taus)